In [1]:
#% What do we use to determine the beginning and end of a reach?


In [2]:
import pandas as pd
from preprocessing.classes import FilterandThresholds, simpleKinematics
import os
import re
import numpy as np
from sympy  import *
import matplotlib.pyplot as plt
import re
#import xarray as xr
velocityChancla = simpleKinematics('tester')
curvePeaks = simpleKinematics('multi_reach')
splitter = FilterandThresholds('Tester single reach')
Multi_indexer = FilterandThresholds('MultiIndex')

path = r'D:/project_deeplabcut/r&d_cam3_2D-CP-2021-07-19/videos'
files = [file for file in os.listdir(path)]

multi_reach= pd.DataFrame()
csvs_animalNum = []
#This section of the code coverts each CSV file into a multiindex dataframe. Then it searches the title of each CSV files
#... to extract animal condition, the number of the reach limb used, and light condition, and whether the reach failed or succeeded.
#... then concatenantes reach CSV and adds variables extracted from the title into a giant dataframe,
for file in files:
    #read csv

    #We're going to have to avoid hidden files with an if statement
    if not file.startswith('.'):
        csv = pd.read_csv('D:/project_deeplabcut/r&d_cam3_2D-CP-2021-07-19/videos/' + file)


        csv = Multi_indexer.MultiIndexMaker(csv)

        csv_complete =Multi_indexer.rexsearch(file, csv)

        multi_reach = pd.concat([multi_reach, csv_complete], ignore_index = True)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8e in position 4: invalid start byte

In [ ]:
#We should add the ability to filter predictions- in this case accurate tracking isnt imperative.


bodypartSelector = FilterandThresholds('TestingThis')

Platform = bodypartSelector.bodypart_selector(multi_reach, 'Platform', likelihoodSelect=True)
plt.plot(Platform['y'])

In [ ]:
Slit = bodypartSelector.bodypart_selector(multi_reach, 'Platform', likelihoodSelect=True)
plt.plot(Slit['Time'], Slit['x'])

import seaborn as sns


In [ ]:
#LIMB LEFT ANALYSIS

#Try one
WristL = bodypartSelector.bodypart_selector(multi_reach, 'WristL' , likelihoodSelect=True)

#We should choose only those reaches that are of left reach.
WristL = WristL[WristL['Limb'] == 'left']
print(WristL)

In [ ]:
#FROM HERE FORWARD THIS COLLECTS REACHES AND SIMPLE KINEMATICS- this can be made into classes.
#Group dataframe by reach and number
WristLbyReach = WristL.groupby(['Reach Number', 'ID'])
keys = WristLbyReach.groups.keys()
x =[]
reaches = []
for key in keys:
    reach = WristLbyReach['x'].get_group(key)
    Time =  WristLbyReach.get_group(key)
    reaches.append(reach)
    x.append(x)
    #print(reaches)

In [ ]:
#Now we're iterating throught he groups.
constantAccVelocity = pd.DataFrame()
WristLExtendCleaned = pd.DataFrame()
cleanedData = pd.DataFrame()
listit = np.arange(100)


In [ ]:
from scipy.ndimage import gaussian_filter1d

#Initiate the simple kinematic dataframes and lists

#Cleaned data frames initialization
Cleanedduration_Frame = pd.DataFrame()
duration_Frame = pd.DataFrame()
Cleanedduration = []
CleanedstrikeNum = []
CleanedIDnum = []
CleanedvelocityAvg = []
cleanCondition = []
cleanPerformance = []

#Dirty data frames initialization
duration = []
strikeNum = []
IDnum = []
velocityAvg = []
Condition = []
performance =[]


for reach, ID in WristLbyReach:
    ID = pd.DataFrame(ID)
    #Calculate the raw velocity and store in dataframe -

    #What if we smooth the ID plot - see how it works...

    ID['smoothed_x'] = gaussian_filter1d(ID['x'], sigma = 2)
    minX = np.argmin(ID['smoothed_x'])

    velocity = velocityChancla.velocity(ID[: minX], 'smoothed_x')
    conditionStr = ID['Condition'].iloc[1]
    performanceStr = ID['performance'].iloc[1]



    #We need to make a peak velocity peak selector - a blanket does not work because the velocity peak max are variable.
    if velocity.size > 0:
        maxPeak = max(velocity)
        minPeak = min(velocity)
        peakHeight_tochoose = round(maxPeak/10)
        troughHeight_tochoose = abs(round(minPeak))
        #print(peakHeight_tochoose)
        #print(troughHeight_tochoose)
    else:
        peakHeight_tochoose = 1

    trough_output_velocity, peak_output_velocity, min_indy, velocity, promineneces_velocity = curvePeaks.curveExtractor(velocity, trough_height=1000, trough_width=None, peak_height=peakHeight_tochoose, peak_width=None, prominences=5, gausFilter=False)
    #print(peak_output_velocity)




    #We need to concatenate all signals
    Cleaned, ExtendCleaned, minIndex = splitter.splitter(signal=ID, velocity_peaks=peak_output_velocity, whichPlane='x' )
    strike, IDNum = reach

    fig, axs = plt.subplots(3)
    fig.suptitle('Comparison to Velocity')
    axs[0].plot(velocity[:minIndex])
    axs[1].plot(ExtendCleaned['x'])
    axs[2].plot(Cleaned['x'])
    axs[1].invert_yaxis()
    axs[2].invert_yaxis()
    axs[0].plot(peak_output_velocity, velocity[peak_output_velocity], "x")
    axs[0].plot(np.zeros_like(velocity), "--", color="gray")
    fig.tight_layout()
    fig.suptitle(reach)




    if len(Cleaned) != len(np.arange(0, (0.003 * len(Cleaned)), 0.003)):
        print('no len match on cleaned')
        Cleaned = Cleaned.iloc[2: , :]
        Cleaned['Time_Aligned'] = np.arange(0, (0.003 * len(Cleaned)), 0.003)
        #add x_vals

        Cleaned['x_vals'] = np.arange(0, len(Cleaned), 1)

        CleanedforVel = Cleaned[['x', 'y', 'Time_Aligned']]
        diff = CleanedforVel.diff()
        coords = [c for c in CleanedforVel if not 'Time_Aligned' in c]
        Cleaned['Velocity'] = np.linalg.norm(diff[coords], axis =1)/diff['Time_Aligned']

        Avgvelocity = np.linalg.norm(CleanedforVel[coords].iloc[-1]-CleanedforVel[coords].iloc[0])/(CleanedforVel['Time_Aligned'].iloc[-1] - CleanedforVel['Time_Aligned'].iloc[0])


        #Calculate duration
        Cleanedduration.append(CleanedforVel['Time_Aligned'].iloc[-1] - CleanedforVel['Time_Aligned'].iloc[0])
        CleanedstrikeNum.append(int(strike))
        CleanedIDnum.append(IDNum)
        CleanedvelocityAvg.append(Avgvelocity)
        cleanCondition.append(conditionStr)
        cleanPerformance.append(performanceStr)
        #Calculate average velocity
    else:
        Cleaned['Time_Aligned'] = np.arange(0, (0.003 * len(Cleaned)), 0.003)
        Cleaned['x_vals'] = np.arange(0, len(Cleaned), 1)

        CleanedstrikeNum.append(int(strike))
        CleanedIDnum.append(IDNum)

        CleanedforVel = Cleaned[['x', 'y', 'Time_Aligned']]
        diff = CleanedforVel.diff()
        coords = [c for c in CleanedforVel if not 'Time_Aligned' in c]
        Cleaned['Velocity'] = np.linalg.norm(diff[coords], axis =1)/diff['Time_Aligned']

        Avgvelocity = np.linalg.norm(CleanedforVel[coords].iloc[-1]-CleanedforVel[coords].iloc[0])/(CleanedforVel['Time_Aligned'].iloc[-1] - CleanedforVel['Time_Aligned'].iloc[0])
        Cleanedduration.append(CleanedforVel['Time_Aligned'].iloc[-1] - CleanedforVel['Time_Aligned'].iloc[0])
        CleanedvelocityAvg.append(Avgvelocity)
        cleanCondition.append(conditionStr)
        cleanPerformance.append(performanceStr)
    if len(ExtendCleaned) != (0.003 * len(ExtendCleaned)):
        print('no len match on Extend')
        ExtendCleaned = ExtendCleaned.iloc[2: , :]
        #print(len(ExtendCleaned))
        ExtendCleaned['Time_Aligned'] = np.arange(0, (3 * len(ExtendCleaned)), 3)
        ExtendCleaned['x_vals'] = np.arange(0, len(ExtendCleaned), 1)

        ExtendCleanedforVel = ExtendCleaned[['x', 'y', 'Time_Aligned']]
        diff = ExtendCleanedforVel.diff()
        coords = [c for c in ExtendCleanedforVel if not 'Time_Aligned' in c]
        #print(CleanedforVel)
        ExtendCleaned['Velocity'] = np.linalg.norm(diff[coords], axis =1)/diff['Time_Aligned']

        duration.append(ExtendCleaned['Time_Aligned'].iloc[-1])



        Avgvelocity = np.linalg.norm(ExtendCleanedforVel[coords].iloc[-1]-ExtendCleanedforVel[coords].iloc[0])/ExtendCleanedforVel['Time_Aligned'].iloc[-1] - ExtendCleanedforVel['Time_Aligned'].iloc[0]

        strikeNum.append(int(strike))
        IDnum.append(IDNum)
        velocityAvg.append(Avgvelocity)
        Condition.append(conditionStr)
        performance.append(performanceStr)


    else:

        ExtendCleaned['Time_Aligned'] = np.arange(0, (0.003* len(ExtendCleaned)), 0.003)
        ExtendCleaned['x_vals'] = np.arange(0, len(ExtendCleaned), 1)

        ExtendCleanedforVel = ExtendCleaned[['x', 'y', 'Time_Aligned']]
        diff = ExtendCleanedforVel.diff()
        coords = [c for c in ExtendCleanedforVel if not 'Time_Aligned' in c]

        ExtendCleaned['Velocity'] = np.linalg.norm(diff[coords], axis =1)/diff['Time_Aligned']
        print('no len match on Extend')
        duration.append(ExtendCleaned['Time_Aligned'].iloc[-1])
        strikeNum.append(int(strike))
        IDnum.append(IDNum)

        xr.align(ExtendCleaned['x'], ExtendCleaned['y'])


        Avgvelocity = np.linalg.norm(ExtendCleanedforVel[coords].iloc[-1]-ExtendCleanedforVel[coords].iloc[0], axis=1)/(WristLExtendCleaned['Time_Aligned'].iloc[-1] - WristLExtendCleaned['Time_Aligned'].iloc[0])

        velocityAvg.append(Avgvelocity)
        Condition.append(conditionStr)
        performance.append(performanceStr)




    cleanedData = pd.concat([cleanedData, Cleaned], ignore_index=True)
    WristLExtendCleaned = pd.concat([WristLExtendCleaned, ExtendCleaned], ignore_index=True)
duration_Frame['duration'] = duration
duration_Frame['Strike'] = strikeNum
duration_Frame['ID'] = IDnum
duration_Frame['AvgVelocity'] = velocityAvg
duration_Frame['Condition'] = Condition
duration_Frame['peformance'] = performance

Cleanedduration_Frame['duration'] = Cleanedduration
Cleanedduration_Frame['Strike'] = strikeNum
Cleanedduration_Frame['ID'] = IDnum
Cleanedduration_Frame['AvgVelocity'] = CleanedvelocityAvg
Cleanedduration_Frame['Condition'] = cleanCondition
Cleanedduration_Frame['performance'] = cleanPerformance


from scipy.ndimage import gaussian_filter1d


In [ ]:
fig = plt.figure(figsize=(10, 8), dpi = 80)
plt.rcParams['figure.figsize'] = (20, 7)
my_pal = {'blind' : 'mediumblue', 'control':'crimson'}
ax = sns.lineplot(x = 'Time_Aligned', y = 'y', hue = 'Condition', data = cleanedData)
plt.xlabel('Time (ms)', fontsize=22)
plt.ylabel('Left Forepaw Vertical Displacement (mm))', fontsize=20)
plt.tick_params(axis = 'both', which = 'major', labelsize = 22)
ax.invert_yaxis()
plt.title('Vertical Displacement over Time ', fontsize=25)
plt.legend(loc=1, prop={'size': 19})
#fig.savefig('/Users/carlospineda/Desktop/TEMP Mono Reaching Code Dev/path_y_time.svg')

In [ ]:
print(WristLExtendCleaned)
fig = plt.figure(figsize=(10, 8), dpi = 80)
plt.rcParams['figure.figsize'] = (20, 7)
my_pal = {'blind' : 'mediumblue', 'control':'crimson'}
ax = sns.lineplot(x = 'Time_Aligned', y = 'x', hue = 'Condition',data = cleanedData)
plt.xlabel('Time (ms)', fontsize=22)
plt.ylabel('Left Forepaw Horizontal Displacement (mm)', fontsize=20)
plt.tick_params(axis = 'both', which = 'major', labelsize = 22)
ax.invert_yaxis()
plt.title('Horizontal Displacement over Time ', fontsize=25)
plt.legend(loc=1, prop={'size': 19})

#fig.savefig('/Users/carlospineda/Desktop/TEMP Mono Reaching Code Dev/path_x_Time.svg')


In [ ]:
# X-Y Plot - non-aligned

fig = plt.figure(figsize=(10, 8), dpi = 80)
Cleaned['x'] = gaussian_filter1d(Cleaned['x'], sigma = 2)
Cleaned['y'] = gaussian_filter1d(Cleaned['y'], sigma = 2)
plt.rcParams['figure.figsize'] = (20, 7)
my_pal = {'blind' : 'mediumblue', 'control':'crimson'}
ax = sns.lineplot(x = 'x', y = 'y', hue = 'Condition', data = cleanedData)
plt.xlabel('Frame', fontsize=20)
plt.ylabel('Left Forepaw Vertical Displacement (mm)', fontsize=20)
plt.tick_params(axis = 'both', which = 'major', labelsize = 20)
ax.invert_yaxis()
plt.title('X-Y Plot - non aligned  ', fontsize=25)
plt.legend(loc=1, prop={'size': 25})
#fig.savefig('/Users/carlospineda/Desktop/TEMP Mono Reaching Code Dev/path_unshifted.svg')


In [ ]:
fig = plt.figure(figsize=(10, 8), dpi = 80)

plt.rcParams['figure.figsize'] = (20, 7)
my_pal = {'blind' : 'mediumblue', 'control':'crimson'}
ax = sns.lineplot(x = 'Time_Aligned', y = 'Velocity', hue = 'Condition',data = WristLExtendCleaned)
plt.xlabel('Time (ms)', fontsize=20)
plt.ylabel('Velocity (mm/s)', fontsize=20)
plt.tick_params(axis = 'both', which = 'major', labelsize = 20)
plt.title('Velocity over time', fontsize=25)
plt.legend(loc=1, prop={'size': 25})
#fig.savefig('/Users/carlospineda/Desktop/TEMP Mono Reaching Code Dev/AggregateVelocity.svg')

In [ ]:
fig = plt.figure(figsize=(10,8), dpi= 80)
p = sns.boxplot(x='Condition', y='AvgVelocity', data=Cleanedduration_Frame)


my_pal = {'blind' : 'mediumblue', 'control':'crimson'}

p.set_xlabel("Condition", fontsize = 22)
p.set_ylabel("Mean Velocity (mm/s)", fontsize = 22)
sns.set(font_scale = 2)
plt.ylim(0, 1000)
plt.title('Mean Velocity', fontsize=25)
plt.ylim()
plt.show()
fig.savefig('/Users/carlospineda/Desktop/TEMP Mono Reaching Code Dev/Average Velocity.svg')
# positions = [0, 1]

In [ ]:
fig = plt.figure(figsize=(10, 8), dpi = 80)
#sns.set(style="darkgrid")
p=sns.boxplot(x="Condition", y="duration", data=Cleanedduration_Frame)
my_pal = {'blind' : 'mediumblue', 'control':'crimson'}

sns.set(font_scale = 3)
# Decoration
p.set_xlabel("Condition", fontsize = 22)
p.set_ylabel("Duration (s)", fontsize = 22)
p.get_xticklabels()
plt.title('Mean Duration', fontsize=25)

plt.ylim(0, 1)
plt.show()
fig.savefig('/Users/carlospineda/Desktop/TEMP Mono Reaching Code Dev/Average Duration.svg')

In [ ]:
#Stats
import scipy.stats as ss
values_per_group = [col for col_name, col in Cleanedduration_Frame.groupby('Condition')['duration']]
ss.ranksums(*values_per_group)

In [ ]:
import scipy.stats as ss
values_per_group = [col for col_name, col in Cleanedduration_Frame.groupby('Condition')['AvgVelocity']]
ss.ranksums(*values_per_group)


In [ ]:
#Fast fourier transform filter
reachSample = cleanedData[cleanedData['Reach Number'] == '1']
plt.rcParams['figure.figsize'] = (20, 7)
my_pal = {'blind' : 'mediumblue', 'control':'crimson'}
ax = sns.scatterplot(x = 'x', y = 'y', hue = 'Condition', style='ID', data = reachSample)
plt.xlabel('Horizontal Position (mm)', fontsize=20)
plt.ylabel('vertical position (mm)', fontsize=20)
plt.tick_params(axis = 'both', which = 'major', labelsize = 20)
ax.invert_yaxis()
ax.invert_xaxis()
plt.legend(loc='upper left', prop={'size': 19})

In [ ]:
reachSample = cleanedData[cleanedData['Reach Number'] == '16']
plt.rcParams['figure.figsize'] = (20, 7)
my_pal = {'blind' : 'mediumblue', 'control':'crimson'}
ax = sns.lineplot(x = 'x', y = 'y', hue = 'Condition', style='ID', data = reachSample)
plt.xlabel('Horizontal Position (mm)', fontsize=20)
plt.ylabel('vertical position (mm)', fontsize=20)
plt.tick_params(axis = 'both', which = 'major', labelsize = 20)
ax.invert_yaxis()
ax.invert_xaxis()
plt.legend(loc='upper left', prop={'size': 19})

In [ ]:
reachSampleExtend = WristLExtendCleaned[WristLExtendCleaned['Reach Number'] == '2']
plt.rcParams['figure.figsize'] = (20, 7)
my_pal = {'blind' : 'mediumblue', 'control':'crimson'}
ax = sns.scatterplot(x = 'x', y = 'y', hue = 'Condition', style='ID', data = reachSampleExtend)
plt.xlabel('Horizontal Position (mm)', fontsize=20)
plt.ylabel('vertical position (mm)', fontsize=20)
plt.tick_params(axis = 'both', which = 'major', labelsize = 20)
ax.invert_yaxis()
ax.invert_xaxis()
plt.legend(loc='upper left', prop={'size': 19})

In [ ]:
reachSampleExtend = WristLExtendCleaned[WristLExtendCleaned['Reach Number'] == '2']
plt.rcParams['figure.figsize'] = (20, 7)
my_pal = {'blind' : 'mediumblue', 'control':'crimson'}
ax = sns.lineplot(x = 'x', y = 'y', hue = 'Condition', style='ID', data = reachSampleExtend)
plt.xlabel('Horizontal Position (mm)', fontsize=20)
plt.ylabel('vertical position (mm)', fontsize=20)
plt.tick_params(axis = 'both', which = 'major', labelsize = 20)
ax.invert_yaxis()
ax.invert_xaxis()
plt.legend(loc='upper left', prop={'size': 19})

In [ ]:
cleanedData['ConditionDummy'] = np.where(cleanedData['Condition'] == "blind", 1, 0)


ax1 = plt.gca( projection='3d')
pnt3D = ax1.scatter(cleanedData['x'],cleanedData['Time_Aligned'],cleanedData['y'], c =cleanedData['ConditionDummy'] ,cmap='seismic')
#ax1.plot(x, y, z, color = 'b')
#pnt3d_slit = ax1.scatter(x_slit, y_slit, z_slit)
#pnt3d_selbow=ax1.scatter(x_elbow,y_elbow, z_elbow)
plt.colorbar(pnt3D)
#pnt3d_platform = ax1.scatter(x_platform, y_platform, z_platform)
#pnt3d_elbow = ax1.scatter(x_elbow, y_elbow, z_elbow)
plt.gca().invert_zaxis()
#plt.gca().invert_yaxis()
#Figure out how to plot this rightside up.

#Figure out how to plot this rightside up.

#cbar = plt.colorbar()
#cbar = plt.colorbar(pnt3d_elbow)
#cbar = plt.colorbar(pnt3d_platform)
# ax1.plot([210, 210], [5,15], [243,243], color='black')
# ax1.plot([220, 220], [11, 11], [210,243], color = 'black')
# ax1.plot([220, 220], [8, 8], [210,243], color = 'black')
# ax1.plot([210, 220], [5, 5], [243,243], color = 'black')
# ax1.plot([210, 220], [15,15], [243,243], color = 'black')
# ax1.plot([220, 220], [5, 15], [243,243], color = 'black')
# ax1.plot([220, 220], [8, 11], [210,210], color = 'black')
# ax1.plot([5, 5], [2.75, 2.75], [-4,-3], color = 'black')
# ax1.plot([5, 5], [1.50, 1.50], [-4,-3], color = 'black')
# ax1.plot([5, 5], [1, 1], [-4,-3], color = 'black')
# ax1.plot([5, 5], [3, 3], [-4,-3], color = 'black')
# ax1.plot([5, 5], [1, 1.75], [-3,-3], color='black')
# ax1.plot([5, 5], [1, 3], [-4,-4], color = 'black')

ax1.set_xlabel('Horizontal Displacement', fontsize = 15)
ax1.set_zlabel('Vertical Displacement)', fontsize = 15)
ax1.set_ylabel('Time (ms)', fontsize = 15)
# plt.xticks(fontsize = 16)
# plt.yticks(fontsize = 16)
# plt.zticks(fontsize = 16)
ax1.tick_params(axis='both', which='major', labelsize=6)
ax1.tick_params(axis='both', which='minor', labelsize=6)
#fig.suptitle('Forepaw Trajectory in 3D', fontsize=24)
#ax1.view_init(0, 60)

plt.show()

In [ ]:
#TESTING

#CALCULATING TANGENTIAL/INSTANT VELOCITY

#Use a single curve for this 11094 control reach 1
reachSample = cleanedData[cleanedData['Reach Number'] == '1']

print(reachSample)